In [2]:
import json
from os.path import split, splitext, join
from pathlib import PosixPath

import cv2 as cv
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from ultralytics import YOLO, settings

from utils import get_file_name_list

# Predict

In [3]:
settings.update({
    'datasets_dir': './output',
    'weights_dir': './output/weights',
    'runs_dir' : './output/runs/',
    'sync': False,
    'api_key': '',
    'clearml': False,
    'comet': False,
    'dvc': False,
    'hub': False,
    'mlflow': False,
    'neptune': False,
    'raytune': False,
    'tensorboard': True,
    'wandb': False,
})
settings

{'settings_version': '0.0.4',
 'datasets_dir': './output',
 'weights_dir': './output/weights',
 'runs_dir': './output/runs/',
 'uuid': '337d8672fe2ab10a5e48a437892eb717fe9ec110fb0f4ea96c9dbc6f915d39d3',
 'sync': False,
 'api_key': '',
 'openai_api_key': '',
 'clearml': False,
 'comet': False,
 'dvc': False,
 'hub': False,
 'mlflow': False,
 'neptune': False,
 'raytune': False,
 'tensorboard': True,
 'wandb': False}

In [88]:
best_device = torch.device('mps')

In [111]:
dataset_meta_path = '../outputs/yolov8/card'
for iter_name in get_file_name_list(dataset_meta_path):
    if iter_name != '10':
        continue
    model_path = join(dataset_meta_path, iter_name, 'weights/best.pt')
    model = YOLO(model_path).to(best_device)
    model.val(
        data='../2_playing_cards/dataset_yolo/data_10.yaml',
        split='test',
        save_json=True,
        conf=0.01,
        iou=0.99,
        half=False,
        max_det=500,
        save_dir=PosixPath(join(dataset_meta_path, iter_name, 'inference_val'))
    )

Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /Users/kamantsev/personal/diploma/2_playing_cards/dataset_yolo/test/labels.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:59<00:00,  1.86s/it]


                   all        500        500      0.207      0.926      0.406      0.262
Speed: 1.0ms preprocess, 56.2ms inference, 0.0ms loss, 32.0ms postprocess per image
Saving /Users/kamantsev/personal/diploma/outputs/yolov8/card/10/inference_val/predictions.json...
Results saved to /Users/kamantsev/personal/diploma/outputs/yolov8/card/10/inference_val


# Convert image_id

In [13]:
with open('../2_playing_cards/dataset_coco/annotations/annotation_test.json') as f:
    coco_gt_file = json.load(f)

In [14]:
file_name_id_map = {splitext(img['file_name'])[0]: img['id'] for img in coco_gt_file['images']}

In [15]:
dataset_meta_path = '../outputs/damo_yolo_ns/cards'
for iter_name in tqdm(get_file_name_list(dataset_meta_path)):
    # if iter_name != '10':
    #     continue
    prediction_file_path = join(dataset_meta_path, iter_name, 'inference_val/predictions.json')
    with open(prediction_file_path) as f:
        coco_prediction_file = json.load(f)

    for item in coco_prediction_file:
        item['category_id'] = 1
        item['image_id'] = file_name_id_map[item['image_id']]
    prediction_file_path, prediction_file_name = split(prediction_file_path)
    prediction_file_name_base, ext = splitext(prediction_file_name)
    with open(join(prediction_file_path, f'{prediction_file_name_base}_converted{ext}'), mode='w+') as f:
        json.dump(coco_prediction_file, f, indent=4)

  0%|          | 0/6 [00:00<?, ?it/s]

KeyError: 4

# NMS suppression

In [50]:
def suppress_nms(coco_test_prediction_file_path):
    with open(coco_test_prediction_file_path) as f:
        coco_prediction_file = json.load(f)
    print(f'initial: {len(coco_prediction_file)}')
    image_id_list = [p['image_id'] for p in coco_prediction_file]
    df_coco_prediction = pd.DataFrame({'id': image_id_list, 'data': coco_prediction_file})
    df_coco_prediction_groups = df_coco_prediction.groupby(by='id')

    confidence_threshold = 0.05
    iou_threshold = 0.7
    coco_prediction_nms_suppressed = []
    for image_id, image_group in tqdm(df_coco_prediction_groups):
        image_data_list = image_group.data.tolist()
        bbox_list = np.asarray([prediction['bbox'] for prediction in image_data_list])
        score_list = np.asarray([prediction['score'] for prediction in image_data_list])
        indices = cv.dnn.NMSBoxes(bbox_list, score_list, confidence_threshold, iou_threshold)
        for index in indices:
            coco_prediction_nms_suppressed.append(image_data_list[index])

    print(f'suppressed: {len(coco_prediction_nms_suppressed)}')
    file_path, file_name = split(coco_test_prediction_file_path)
    file_name, ext = splitext(file_name)
    with open(join(file_path, f'{file_name}_nms{ext}'), mode='w+') as f:
        json.dump(coco_prediction_nms_suppressed, f, indent=4)

In [47]:
dataset_meta_path = '../outputs/damo_yolo_ns/'
for ds_name in get_file_name_list(dataset_meta_path):
    # if ds_name != 'cards':
    #     continue
    for iter_name in get_file_name_list(join(dataset_meta_path, ds_name)):
        # if iter_name != '10':
        #     continue
        # suppress_nms(join(dataset_meta_path, ds_name, iter_name, 'damoyolo_tinynasL18_Nm/inference/coco_val/bbox.json'))
        suppress_nms(join(dataset_meta_path, ds_name, iter_name, r'damoyolo_tinynasL18_Ns_test2/inference/coco_test/bbox.json'))
        # suppress_nms(join(dataset_meta_path, ds_name, iter_name, 'inference_val/predictions_converted.json'))

initial: 216500


  0%|          | 0/433 [00:00<?, ?it/s]

suppressed: 1994
initial: 216500


  0%|          | 0/433 [00:00<?, ?it/s]

suppressed: 4979
initial: 216500


...

suppressed: 3475
initial: 216500


  0%|          | 0/433 [00:00<?, ?it/s]

suppressed: 7172
initial: 216500


  0%|          | 0/433 [00:00<?, ?it/s]

suppressed: 13765


# Custom metric

In [35]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou


def xywh2xyxy(bbox_xywh):
    x, y, w, h = bbox_xywh

    return [x, y, x + w, y + h]


def get_custom_scores(coco_gt, coco_prediction, threshold=0.1, threshold_iou=0.5):
    tp, fp, fn = 0, 0, 0
    for annotation_descriptor in tqdm(coco_gt['annotations']):
        image_id = annotation_descriptor['image_id']  # we are sure to have only 1 gt annotation for each image
        bbox_gt = annotation_descriptor['bbox']
        bbox_gt = xywh2xyxy(bbox_gt)
        image_predictions = list(sorted([p for p in coco_prediction if p['image_id'] == image_id], key=lambda it: it['score'], reverse=True))[:100]
        any_prediction = False
        for prediction_descriptor in image_predictions:
            if prediction_descriptor['score'] < threshold:
                continue
            bbox_predicted = xywh2xyxy(prediction_descriptor['bbox'])
            iou = bb_intersection_over_union(bbox_gt, bbox_predicted)
            if iou > threshold_iou and not any_prediction:
                tp += 1
                any_prediction = True
            else:
                fp += 1
        if not any_prediction:
            fn += 1

    return tp, fp, fn

In [36]:
coco_test_gt_file_path = '../2_playing_cards/dataset_coco/annotations/annotation_test.json'
with open(coco_test_gt_file_path) as f:
    coco_gt_file = json.load(f)

In [44]:
dataset_meta_path = '../outputs/damo_yolo_ns/cards'
for folder_name in sorted(get_file_name_list(dataset_meta_path), reverse=True):
    # if folder_name != '10':
    #     continue
    prediction_coco_path = join(dataset_meta_path, folder_name, 'damoyolo_tinynasL18_Ns_test/inference/coco_test/bbox.json')
    # prediction_coco_path = join(dataset_meta_path, folder_name, 'inference_val/predictions_converted_nms.json')
    # prediction_coco_path = './output/runs/detect/val2/prediction_custom.json'
    with open(prediction_coco_path) as f:
        coco_prediction_file = json.load(f)
    for item in coco_prediction_file:
        if item['score'] < 0.01:
            print(1)
    print(folder_name)
    print(len(coco_prediction_file))
    print()
    # tp, fp, fn = get_custom_scores(coco_gt_file, coco_prediction_file, threshold=0.6)
    # print(f'total predictions: {tp + fp}')
    # print(f'precision: {tp / (tp + fp)}')
    # print(f'recall: {tp / (tp + fn)}')

10
216500

025
216500

01
216500

0075
216500

005
216500

0025
216500



# Coco-metrics

In [28]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from IPython.utils import io

In [29]:
def get_coco_metrics(gt_file_path, prediction_file_path):
    with io.capture_output() as _:
        cocoGt = COCO(gt_file_path)
        cocoDt = cocoGt.loadRes(prediction_file_path)
        cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
        cocoEval.evaluate()
        cocoEval.accumulate()
    cocoEval.summarize()

In [45]:
annotation_gt_path = '../2_playing_cards/dataset_coco/annotations/annotation_test.json'
dataset_meta_path = '../outputs/damo_yolo_ns/cards'
for folder_name in sorted(get_file_name_list(dataset_meta_path), reverse=True):
    if folder_name != '10':
        continue
    # prediction_coco_path = join(dataset_meta_path, folder_name, 'damoyolo_tinynasL18_Ns/inference/coco_val/bbox_nms.json')
    prediction_coco_path = join(dataset_meta_path, folder_name, 'damoyolo_tinynasL18_Ns_test2/inference/coco_test/bbox_nms.json')
    # prediction_coco_path = join(dataset_meta_path, folder_name, 'inference_val/predictions_converted_nms.json')
    print(f'Prediction for: {folder_name}')
    get_coco_metrics(annotation_gt_path, prediction_coco_path)

Prediction for: 10
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.550
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.887
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.662
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.634
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.656
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.574
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.660
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
